# Assignment 3 - Building a Custom Visualization

---

In this assignment you must choose one of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use matplotlib so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented task-driven visualizations: allowing users to make better, more confident decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. ([video](https://www.youtube.com/watch?v=BI7GAs-va-Q))


In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number votes in a given district or riding for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="readonly/Assignment3Fig1.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from (Ferreira et al, 2014).</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="readonly/Assignment3Fig2c.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from (Ferreira et al. 2014). Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with only three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.


**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [ ]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gspec
import matplotlib.animation as anime
import scipy.stats as stats

from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
# Use the following data for this assignment:

import pandas as pd
import numpy as np

np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650)], 
                  index=[1992,1993,1994,1995])
df

In [ ]:
def coverageFunction(minima,maxima):
    colors_list = []
    means = np.asarray(meanRows)
    for i in range(0,len(y_errors)):
        y_max = means[i]+y_errors[i]
        y_min = means[i]-y_errors[i]
        if minima > y_max:
            colors_list.append(graphMap.to_rgba(0))
        elif maxima < y_min:
            colors_list.append(graphMap.to_rgba(0))
        elif (maxima >= y_max) & (minima <= y_min):
            colors_list.append(graphMap.to_rgba(1))
        elif (minima <= y_min) & (maxima <= y_max):
            colors_list.append(graphMap.to_rgba((maxima - y_min)/(y_max - y_min)))
        elif (minima > y_min) & (maxima < y_max):
            colors_list.append(graphMap.to_rgba((maxima - minima)/(y_max - y_min)))
        else:
            colors_list.append(graphMap.to_rgba((y_max - minima)/(y_max - y_min)))
    return colors_list

In [ ]:
def exec_scroll(minima, maxima):
    top_bargraph.cla()
    barColors = coverageFunction(minima, maxima)
    top_bargraph.bar(df.index,meanRows,yerr = y_errors,color=barColors)
    global min_line,max_line
    min_line = top_bargraph.axhline(minima,0,1,color="orange",linestyle ='dashdot')
    max_line = top_bargraph.axhline(maxima,0,1,color="orange",linestyle ='dashdot')
    top_bargraph.text(1993.3, minima,"y_min = "+str(round(minima,2)), bbox=dict(facecolor='orange', alpha=0.5), fontsize=7)
    top_bargraph.text(1993.3,maxima, "y_max = "+str(round(maxima,2)), bbox=dict(facecolor='orange', alpha=0.5), fontsize=7)
    top_bargraph.set_xticks(df.index)
    top_bargraph.set_title("year v/s distribution means with confidence intervals")

### Option 1 : use interactive scrollers for the range with animate

In [ ]:
a = np.NaN
def updateList(current,minima,maxima):
    if current == 0:
        try:
            exec_scroll(minima, maxima)
            a.event_source.stop()
        except:
            top_bargraph.set_title("error occured")
    else:
        top_bargraph.set_title(current)
        a.event_source.stop()

In [ ]:
def add_interactivescroller() :
    interact(scroller_animation, range_min=widgets.IntSlider(min=0,max=55000,step=100,value=10000),
                                 range_max=widgets.IntSlider(min=min_line.get_ydata()[0],max=55000,step=100,value=15000))

In [ ]:
def scroller_animation(range_min, range_max):
    global a,fig
    a = anime.FuncAnimation(fig, updateList, fargs=(range_min,range_max))
    plt.show()



### option 2 : click on graph to run interative

In [ ]:
def onclick(event):
    y_val = event.ydata
    if np.abs(y_val - max_line.get_ydata()[0]) > np.abs(y_val - min_line.get_ydata()[0]):
        exec_scroll(y_val, max_line.get_ydata()[0])

    else:
        exec_scroll(min_line.get_ydata()[0], y_val)
    
plt.gcf().canvas.mpl_connect('button_press_event', onclick)
            

In [ ]:
meanRows= np.mean(df, axis=1)
y_errors= stats.sem(df,axis =1 )
critical_val_norm = stats.norm.ppf(0.95)
y_errors= y_errors*critical_val_norm

fig = plt.figure()
grid = gspec.GridSpec(6,1,hspace= 0.7)
top_bargraph = plt.subplot(grid[0:5,0])
bottom_colorbar = plt.subplot(grid[5,0])

colorbar = mpl.colorbar.ColorbarBase(bottom_colorbar,
                          orientation='horizontal', 
                          cmap=mpl.cm.Reds,norm = mpl.colors.BoundaryNorm(boundaries=np.arange(0,1.1,0.1), ncolors=256)
                         )
colorbar.set_ticks(np.arange(0,1.1,0.1))

colorMap = mpl.cm.Reds
graphMap = mpl.cm.ScalarMappable(norm = mpl.colors.Normalize(vmin=0, vmax=1),cmap=colorMap)

top_bargraph.bar(df.index,meanRows,yerr = y_errors,color= graphMap.to_rgba(0))
top_bargraph.set_xticks(df.index)

min_line = top_bargraph.axhline(10000,0,1,color="orange",linestyle ='dashdot')
max_line = top_bargraph.axhline(15000,0,1,color="orange",linestyle ='dashdot')
top_bargraph.set_facecolor("lightblue")
top_bargraph.set_title("year v/s distribution means with confidence intervals")


In [ ]:
add_interactivescroller()


In [ ]:
plt.savefig("visualization.jpg",bbox_inches="tight")

In [ ]:
%matplotlib inline